In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

# Create vector database
class VectorStore:
    """Class to create VectorDB"""
    def __init__(self, data_path, db_path):
        """
        Initialize the variables for VectorStore.
        """
        self.data_path = data_path
        self.db_path = db_path
        
    def create_vector_db(self):
        """
        function to build vector DB.
        """
        DATA_PATH = self.data_path
        DB_FAISS_PATH = self.db_path
        loader_pdf = DirectoryLoader(DATA_PATH,
                                 glob=f'*.pdf',
                                 loader_cls=PyPDFLoader)

        loader_text = DirectoryLoader(DATA_PATH,
                                 glob=f'*.txt',
                                 loader_cls=TextLoader)

        documents_pdf = loader_pdf.load()
        documents_text = loader_text.load()
        documents = documents_pdf + documents_text
        for i in documents:
            i.page_content = i.page_content.replace(' . ','').replace('\n',' ')
            i.page_content = re.sub(r'\.+', ".", i.page_content)
            i.page_content = ' '.join(i.page_content.split())

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                       chunk_overlap=50)
        texts = text_splitter.split_documents(documents)

        embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2',
                                           model_kwargs={'device': 'cuda'})

        db = FAISS.from_documents(texts, embeddings)
        db.save_local(DB_FAISS_PATH)

if __name__ == "__main__":
    data_path = '/home/chirayu.tripathi/hackathon/'
    db_path = '/home/chirayu.tripathi/hackathon/vectorstore/db_faiss'
    obj = VectorStore(data_path, db_path)
    obj.create_vector_db()

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
import chainlit as cl
from langchain.llms import HuggingFacePipeline
from torch import cuda, bfloat16
import transformers
import gradio as gr

class VerizonAI:
    """ Class VerizonAI to create the chatbot object """
    def __init__(self, db_path):
        """
        Initialize the class object.
        """
        self.DB_FAISS_PATH = '/home/chirayu.tripathi/hackathon/vectorstore/db_faiss'

        self.custom_prompt_template = """[INST] You are a Verizon company's chatbot, Only use the following pieces of context to answer the user's question. If the answer is not present in context, just say that you don't know and display the following link "https://www.verizon.com/support/residential/contact-us/contactuslanding.htm", don't try to make up an answer.[/INST]

        Context: {context}
        Question: {question}
        answer: 
        """
        self.generate_text = self.get_pipeline()
        self.qa_result = self.qa_bot()
        
    def get_pipeline(self):
        # model_id = 'meta-llama/Llama-2-7b-chat-hf'
        model_id = 'mistralai/Mistral-7B-Instruct-v0.1'
        device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'


        bnb_config = transformers.BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=bfloat16
        )

        hf_auth = '' #not required for mistral, but required for llama-2
        model_config = transformers.AutoConfig.from_pretrained(
            model_id,
            use_auth_token=hf_auth
        )

        model = transformers.AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            config=model_config,
            quantization_config=bnb_config,
            device_map='auto',
            use_auth_token=hf_auth
        )

        model.eval()

        tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_id,
        use_auth_token=hf_auth
        )

        print(f"Model loaded on {device}")

        generate_text = transformers.pipeline(
        model=model, 
        tokenizer=tokenizer,
        return_full_text=True,  # langchain expects the full text
        task='text-generation',
        # we pass model parameters here too
        # stopping_criteria=stopping_criteria,  # without this model rambles during chat
        temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
        max_new_tokens=512,  # max number of tokens to generate in the output
        repetition_penalty=1.1  # without this output begins repeating
        )
        return generate_text
    
    def set_custom_prompt(self):
        """
        Prompt template for QA retrieval for each vectorstore
        """
        prompt = PromptTemplate(template=self.custom_prompt_template,
                                input_variables=['context', 'question'])
        return prompt



    def retrieval_qa_chain(self, llm, prompt, db):
        """ Create the QA retrieval chain from langchain. """
        qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                           chain_type='stuff',
                                           retriever=db.as_retriever(search_kwargs={'k': 3}),#, search_type="mmr"),
                                           return_source_documents=True,
                                           # memory = memory
                                           chain_type_kwargs={'prompt': prompt}
                                           )

        return qa_chain
    
    def load_llm(self):
        """ Load the model using the huggingface pipeline. """
        llm = HuggingFacePipeline(pipeline=self.generate_text)
        return llm

    
    def qa_bot(self):
        """Setup the QA bot"""
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2",
                                           model_kwargs={'device': 'cuda'})
        db = FAISS.load_local(self.DB_FAISS_PATH, embeddings)
        llm = self.load_llm()
        qa_prompt = self.set_custom_prompt()
        qa = self.retrieval_qa_chain(llm, qa_prompt, db)

        return qa

    
    def final_result(self,message,history):
        """Function to generate the result"""
        response = self.qa_result({'query': message})
        return response['result']
    
if __name__ == "__main__":
    bot = VerizonAI(db_path = '/home/chirayu.tripathi/hackathon/vectorstore/db_faiss')
    gr.ChatInterface(bot.final_result, title="VerAIzon Bot").launch(share=True)


/home/chirayu.tripathi/venv/lib/python3.8/site-packages/transformers/models/auto/configuration_auto.py:1020: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/chirayu.tripathi/venv/lib/python3.8/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/chirayu.tripathi/venv/lib/python3.8/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/chirayu.tripathi/venv/lib/python3.8/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model loaded on cuda:0
2023-10-15 16:04:35 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L12-v2
Running on local URL:  http://127.0.0.1:7861


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://5bf6106a637dd6a0a3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [15]:
# model
bot = VerizonAI(db_path = '/home/chirayu.tripathi/hackathon/vectorstore/db_faiss')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model loaded on cuda:0
2023-10-15 13:54:02 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L12-v2


In [16]:
bot.final_result('What is the procedure to listen to my messages?','ff')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'1. Call Home Voice Mail as described in “Accessing Your Mailbox.” \n        2. Press 1 at the Main Menu to hear your messages. \n        3. After you hear each message, you can:\n            * Press 1 to replay the message.\n            * Press 2 to save it.\n            * Press 3 to erase it.\n            * Press 4 to reply to a message left by another mailbox subscriber.\n            * Press 5 to send a copy to another mailbox.'

In [ ]:
qa_result = qa_bot()
response = qa_result({'query': 'What is the procedure to listen to my messages, provide in bullet points'})
response

In [161]:
qa_result = qa_bot()
response = qa_result({'query': 'what are the deals on iphones?'})
response

2023-10-15 13:24:57 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L12-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'query': 'what are the deals on iphones?',
 'result': '\nWe have several deals available on iPhones. One offer includes up to $999.99 device payment or full retail purchase with a new smartphone line on the Unlimited Ultimate plan required. With this offer, you can receive less up to $999.99 promo credit applied over 36 months; promo credit ends if eligibility requirements are no longer met; 0% APR. Another offer includes trading in your current device and saving $830 on the purchase of a new iPhone. Additionally, we have a deal where you can save on both the iPhone and Apple Watch when you buy them together. Please visit our website for more information on these offers and how to redeem them.',
 'source_documents': [Document(page_content="offer.Up to $999.99 device payment or full retail purchase w/ new smartphone line on Unlimited Ultimate plan req'd. Less up to $999.99 promo credit applied over 36 mos.; promo credit ends if eligibility req’s are no longer met; 0% APR. end of naviga

In [3]:
qa_result = qa_bot()
response = qa_result({'query': 'Do you provide iphones cases?'})
# response

NameError: name 'qa_bot' is not defined

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [80]:
response

{'query': 'Do you provide iphones cases?',
 'result': 'I am sorry, I do not have access to the current sale information. However, you can check out our website for the latest deals on cases and chargers for iPhones. Here is a link to our website where you can find more information: <https://www.verizon.com/shop/accessories/iphone-cases>',
 'source_documents': [Document(page_content='Sale: Deals on Cases, Chargers, More | Verizon Accessibility Resource Center Skip to main content Personal Business Stores Español Shop Shop Shop Close Shop Shop all Deals Deals Deals Shop all deals Free phones My offers Smartphones Fios Home Internet Bring your own device Accessories Refer a Friend Refer a Friend Devices Devices Devices Smartphones 5G phones Certified pre-owned phones Featured smartphones Featured smartphones Featured smartphones Apple iPhone 15 Pro Apple iPhone 15 Samsung', metadata={'source': '/home/chirayu.tripathi/hackathon/verizon_101.txt'}),
  Document(page_content='Accessories: Char

In [163]:
qa_result = qa_bot()
response = qa_result({'query': 'How to access my mailbox, if I am not at home?'})

2023-10-15 13:27:29 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L12-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [164]:
response

{'query': 'How to access my mailbox, if I am not at home?',
 'result': 'To access your mailbox when away from home: 1. Dial your Access Number, with area code when necessary. 2. Wait for an answer, then press # 3. Enter your mailbox number, (your 7 or 10 digit home telephone number depending on your area). 4. Enter your Passcode to reach the Home Voice Mail Main Menu.',
 'source_documents': [Document(page_content='mailbox parameters. Once the message limit has been reached, you cannot receive new messages until you erase some of your stored messages. Accessing Your Mailbox To listen to your messages or to access other Home Voice Mail options, you will first need to dial into your mailbox. To access your mailbox from your home telephone: 1. Dial your Access Number. 2. Enter your Passcode to reach the Home Voice Mail Main Menu. To access your mailbox when away from home: 1. Dial your Access Number, with', metadata={'source': '/home/chirayu.tripathi/hackathon/north_hvm_ug_h2069.pdf', 'pag

In [17]:
import gradio as gr
gr.ChatInterface(bot.final_result).launch(share=True)

Running on local URL:  http://127.0.0.1:7860


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://8be408bbc4d8fe746b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]